In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [ ]:
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
random_seed = 42

from models import LM, LMLoss
from data import LM_Dataset
from trainer_ml import LMTrainer
from utils import loaders_from_dataset, visualize_embeddings, save_model

In [ ]:
dataset = LM_Dataset('datasets/wmt14_en_ger/train _small.en')
loaders = loaders_from_dataset(dataset, batch_size=32, val_perc_size=0.05)

In [ ]:
%tensorboard --logdir=data

In [ ]:
epochs = 500
model = LM(vocab_size=len(dataset.Ind2word), emb_dim=50, rnn=torch.nn.GRU)
optim = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.001)
params_trainer = {
    'model': model,
    'loaders': loaders,
    'criterion': LMLoss(),
    'optim': optim,
    'val_step': 5,
    'scheduler': torch.optim.lr_scheduler.CosineAnnealingLR(optim, epochs),
    'verbose': False,
    'device': device,
#     'params_clearml': params_clearml,
    'is_tensorboard': True
}

trainer = LMTrainer(**params_trainer)

In [ ]:
trainer.loop(0, epochs, exp_name='GRU_wmt2014_small')

In [ ]:
word2Ind = {word: i for i, word in dataset.Ind2word.items()}

chosen_words = [
    "świetny", "głupi", "wspaniały", "spoko", "ekstra",
    "słodki", "nudny",   "zły", 
    "artyleria", "generał", 'porucznik', "sierżant", "wojsko",
#     "dobrze"
] 
chosen_idxs = [word2Ind[word] for word in chosen_words]

visualize_embeddings(model.embs.weight.detach().cpu().numpy(), chosen_idxs, dataset.Ind2word)

## Length of sentence distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sent_len = np.array([len(sent) for sent in dataset.indexed_data])

In [ ]:
plt.hist(sent_len, bins=100);

In [ ]:
print(sent_len.mean(), sent_len.std(), sent_len.min(), sent_len.max())

Uwagi:
- Czy nie powinienem brac P(x_t|x_1, x_2,...,x_{t-1}) dopiero od pewnego t > 2? (t > 5)?
- Zbudować ewaluacje, 
- Poprawić trainera na guthubie (os.makedirs, .pth)

In [ ]:
from utils import load_model

In [ ]:
model = LM(vocab_size=len(dataset.Ind2word), emb_dim=50, rnn=torch.nn.LSTM)
load_model(model, 'data/LSTM/2022-04-01_00-20-06/checkpoints/_epoch_199.pth')